<a href="https://colab.research.google.com/github/Tiger-A/GoogleCollab/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_25_%D0%BC%D0%B0%D1%8F_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%9A%D0%9A_%D0%B8_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE_%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BA%D1%82%D0%BE%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек. Сервисные функции
!pip -q install --upgrade tiktoken
!pip -q install langchain openai chromadb
!pip -q install gspread oauth2client

from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

import os
import openai
import tiktoken
import re


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class GPT():
  def __init__(self):
    pass

  @classmethod
  def set_key(cls):
      password_input = widgets.Password(
          description='Введите пароль:', 
          layout=widgets.Layout(width='500px'),
          style={'description_width': 'initial', 'white-space': 'pre-wrap', 'overflow': 'auto'})
      login_button = widgets.Button(description='Авторизация')
      output = widgets.Output()

      def on_button_clicked(_):
          with output:
              openai.api_key = password_input.value
              os.environ["OPENAI_API_KEY"] = openai.api_key
              print(f'{bcolors.OKGREEN}{bcolors.BOLD}Ключ сохранен!{bcolors.ENDC}')
              password_input.layout.display = 'none'
              login_button.layout.display = 'none'

      login_button.on_click(on_button_clicked)
      display(widgets.VBox([password_input, login_button, output]))

  def load_search_indexes(self, url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return self.create_embedding(text)

  def load_prompt(self, 
                  url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return f'{text}'


  def create_embedding(self, data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    for chunk in splitter.split_text(data):
      source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

  def answer(self, system, topic, temp = 1):    
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content

  def num_tokens_from_messages(self, messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
        
  def insert_newlines(self, text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


  def dialog(self):
    user = ''
    dialog = ''

    print(f'{bcolors.OKBLUE}{bcolors.BOLD}С чем связан ваш интерес к искусственному интеллекту?{bcolors.ENDC}')

    while user.lower() not in ['stop', 'exit', 'выход']:
      user = input('Клиент: ')
      if (user == 'Stop'): break

      dialog += '\n\n' + 'Клиент: ' + user
      add_dialog = gpt.answer(expert_prompt, user)
        
      dialog += '\n\n' + 'Менеджер: ' + add_dialog      
      print(f'\n{bcolors.OKBLUE}{bcolors.BOLD}Менеджер:{bcolors.ENDC} {self.insert_newlines(add_dialog)}')
      report = gpt.answer(validation_prompt, dialog)
      answer = gpt.answer(action_prompt, report)

      print(f'\n{bcolors.OKGREEN}{bcolors.BOLD}Отчёт системы:\n {bcolors.ENDC}{report}')
      print(f'\n{bcolors.HEADER}{bcolors.BOLD}Менеджер: {bcolors.ENDC}{self.insert_newlines(answer)}\n\n')

    return dialog

  def answer_index(self, system, topic, search_index, temp = 1, verbose = 0):
    
    #Выборка документов по схожести с вопросом 
    docs = search_index.similarity_search(topic, k=5)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{self.num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

    # return completion

  def get_chatgpt_ansver3(self, system, topic, temp = 1):
    
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.5/846.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
gpt = GPT()

In [ ]:
GPT.set_key()

<h3><b>КК zoom презентации менеджера</b></h3>

In [ ]:
#Методичка
zoom_index = gpt.load_search_indexes('https://docs.google.com/document/d/1MuQ02a3Kz6ysDN43SM5YrURS2Mg2trLNrM_TyIElZLQ/edit?usp=sharing')


 ===========================================: 
Количество токенов в документе : 18071
ЦЕНА запроса: 0.007228400000000001  $


In [ ]:
zoom_promt = '''Инструкция по оценке диалога менеджера по продажам с клиентом

Перед тобой диалог менеджера по продажам с клиентом
Тебе надо проверить несколько критериев и заполнить отчёт
Не пиши общее сообщение, только отчёт по форме, форма отчёта будет в конце

Что надо проверить
1. Говорил ли клиент о потребности в трудоустройстве
2. Говорил ли клиент о потребности создать AI проект
3. Говорил ли клиент возражение о стоимости обучения
4. Говорил ли клиент возражение о времени на обучение

Форма отчёта
1. Потребность о трудоустройстве - было или нет
2. Потребность о создании AI проекте - было или нет
3. Возражение о стоимости - было или нет
4. Возражение о времени обучения - было или нет

Заполни отчёт и пришли в качестве ответа, коротко и ёмко'''

In [ ]:
gpt.answer_index(
    zoom_promt,
    'Пришли отчет',
    zoom_index
)

ОТВЕТ : 
  1. Потребность о трудоустройстве - была 2. Потребность о создании AI проекта - была 3. Возражение о стоимости - не было 4. Возражение о времени обучения - не было


In [ ]:
gpt.answer_index(
    zoom_promt,
    'Пришли отчет',
    zoom_index
)

ОТВЕТ : 
  Отчет: 1. Потребность в трудоустройстве - было 2. Потребность в создании AI проектах - было 3. Возражение о стоимости - не было 4. Возражение о времени обучения - не
 было


In [ ]:
zoom_promt_2 = '''Ты менеджер контроля качества, твоя задача анализировать диалоги менеджеров по продажам с клиентами и готовить отчеты.
Компания продает курсы по AI разработке. 
Перед тобой текст диалога сделанный с помощью распознавания речи из записи zoom презентации.
Из-за машинного распознавания речи, в тексте могут быть ошибки распознавания, учитывая это.
Твоя задача делать отчеты по данному диалогу по запросам пользователя.
Составляй вопросы максимально точно по диалогу, не придумывай ничего от себя. 
Текст диалога: '''

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, было ли от менеджера прямое предложение клиенту купить курс',
    zoom_index
)

ОТВЕТ : 
  Из представленного диалога нельзя однозначно сказать, было ли прямое предложение клиенту купить курс. Менеджер обсуждал с клиентом различные тарифы, подробности обучения
 и возможности трудоустройства после курса. Однако, наиболее близким предложением к покупке курса были слова менеджера: "Как По тарифам? Какой больше понравился?" Однако,
 это не совсем прямое предложение купить курс, а скорее вопрос о том, какую программу выбрать.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, было ли от менеджера прямое предложение клиенту купить курс. Если да, процитируй дословно. В начале выведи прямой ответ "Да" или "Нет", а только потом уже общие рассуждения',
    zoom_index
)

ОТВЕТ : 
  Отчет: Да В ходе диалога менеджер напрямую предлагал клиенту купить курс, ссылаясь на конкретные тарифы и условия обучения. Один из отрывков с таким предложением
 выглядит следующим образом: "Как По тарифам? Какой больше понравился?" Также менеджер предлагал рассмотреть дополнительные услуги и планировал провести пробное занятие,
 чтобы привлечь клиента к покупке курса. Однако, несмотря на торговые мотивы менеджера, клиент поддерживал дискуссию и выражал сомнения в отношении покупки курса.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    '''Напиши отчет, было ли от менеджера прямое предложение клиенту купить курс. Если да, процитируй дословно. 
    В начале выведи прямой ответ "Да" или "Нет", а только потом уже общие рассуждени.
    ''',
    zoom_index
)

ОТВЕТ : 
  Да, было прямое предложение клиенту купить курс. Менеджер сказал: "Вот тарифа проект под ключ. У вас шестьдесят индивидуальных консультаций шестьдесят часов, то есть вот
 тот проект, который будете выполнять, можно просто упаковать, будет от и до. За этих индивидуальных консультациях может быть даже и не только про него. Вы же будете и По
 поводу обучения могу разговаривать. Вдруг там что то будет вам не понятно или не ясно?" Фразы, которые можно считать прямым предложением купить: - Ну что, давайте
 забронируем место на обучении? - Как смотрите на то, чтобы начать обучение уже завтра? - Давайте прямо сейчас оформими рассрочку на обучение Фразы, которые не являются
 прямым предложением купить: - Ну что думаете про обучение? - Когда планируете начать обучение? - Какой тариф вам подходит?


In [ ]:
zoom_promt_9 = '''Ты менеджер контроля качества, твоя задача анализировать диалоги менеджеров по продажам с клиентами и готовить отчеты.
Компания продает курсы по AI разработке. 
Перед тобой текст диалога сделанный с помощью распознавания речи из записи zoom презентации.
Из-за машинного распознавания речи, в тексте могут быть ошибки распознавания, учитывая это.

Напиши отчет, было ли от менеджера прямое предложение клиенту купить курс. Если да, процитируй дословно. 
    В начале выведи прямой ответ "Да" или "Нет", а только потом уже общие рассуждени.
    Какие фразы можно считать прямым предложение купить
    - Ну что, давайте забронируем место на обучении?
    - Как смотрите на то, чтобы начать обучение уже завтра?
    - Давайте прямо сейчас оформими рассрочку на обучение
    И другие подобные прямые и напористые фразы

    Какие фразы не являются прямым предложение купить
    - Ну что думаете про обучение?
    - Когда планируете начать обучение?
    - Какой тариф вам подходит
    И другие подобные фразы, где менеджер не проявляет напористости

Составляй ответы максимально точно по диалогу, не придумывай ничего от себя.

Текст диалога: '''

In [ ]:
gpt.answer_index(
    zoom_promt_9,
    '''Напиши отчет''',
    zoom_index
)

ОТВЕТ : 
  Документ №1: Нет прямого предложения купить курс. Документ №2: Да, было прямое предложение купить курс. Менеджер предложил клиенту выбрать 20-30 вакансий и сделать для
 них тестовые задания, проходить собеседование у них и в дальнейшем предлагают работу. Документ №3: Нет прямого предложения купить курс. Документ №4: Нет прямого
 предложения купить курс. Документ №5: Нет прямого предложения купить курс. В общем, из всех документов только один содержит прямое предложение купить курс. Были указаны
 фразы, которые можно считать прямым предложением, и которые не являются таковыми. Однако, следует отметить, что наличие прямого предложения зависит не только от фраз, но
 и от контекста, темы разговора и тонов голосов собеседников.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, какие были потребности названы клиентом',
    zoom_index
)

ОТВЕТ : 
  Отчет о потребностях клиента: Проанализировав диалог между менеджером и клиентом, можно выделить следующие потребности клиента: 1. Создание персонального шедевра через
 использование некоторых навыков, таких как вязание крючком. Клиент хочет определить свой стиль и подобрать уникальное изделие, которое будет модным. Также клиент
 интересовался возможностью оптимизации процесса создания персонального шедевра. 2. Понимание формата обучения. Клиент хотел привлекательный, интересный и необычный
 формат обучения, отличающийся от обычных методов. 3. Трудоустройство. Клиент интересовался процессом трудоустройства после обучения и возможностью увеличить свой опыт до
 необходимого для компаний уровня. Клиенту было предложено двадцать тридцать вакансий на выбор, с возможностью сделать для них тестовые задания и проходить собеседование.
 4. Использование искусственного интеллекта. В диалоге затрагивалась тема использования искусственного интеллекта в различных проектах, таких как отсле

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, почему клиенту вообще интересно изучение AI',
    zoom_index
)

ОТВЕТ : 
  Клиент проявляет интерес к AI в связи с поиском современных технологий, которые могут быть применимы в ее работе, она хочет разобраться в принципах функционирования
 искусственного интеллекта для того, чтобы использовать их в повседневной практике. В ходе диалога тема искусственного интеллекта неоднократно поднималась и клиент
 проявляла интерес к обучению на данной тематике, а также узнавала о возможностях применения AI в конкретных сферах деятельности.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, какие возражения назвал клиент',
    zoom_index
)

ОТВЕТ : 
  Клиент не высказывал конкретных возражений, но задавал вопросы о цене и тарифах, а также интересовался, какие программы обучения предлагаются и возможности
 трудоустройства после курсов. Также клиент выразил свои сомнения относительно своей способности обучаться технологиям и проектам в сфере AI разработки, поскольку ранее
 не имел опыта работы в этой области. Тем не менее, клиент выразил интерес и готовность начать обучение с базовых программ и определенных тарифов обучения.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    '''Напиши отчет, какие возражения назвал клиент.
    Возражения бывают:
    - про стоимости и деньги, что курс слишком дорогой и у клиента нет такого количества денег
    - про время на обучение, что клиент очень занят и не будет успевать учиться
    - про негативные отзывы о компани и обучении
    - возражение про статус компании, что мы действительно авторитетная и весомая компания с сильным обученим
    - про то, что у клиента нет опыта программирования и матетики, что у него не получится освоить курс
    - про то, что клиент сначала хочет попробовать пробные занятия и только потом принимать решение
    Бывают и другие возражения, не только эти
    Возражение - это любые слова клиента, которые останавливают его от покупки
    Отнесись к возражениям критично, всё, что как-либо похоже на сомнения или возражения клиента, засчитай в возражения
    Выпиши возражения списком с номерами.
    К каждому возражению добавь цитаты, как клиент дословно произносил возражение.
    К каждому возражению добавь твои предложения, как можно закрыть это возражение, чтобы клиент купил обучение.
     ''',
    zoom_index
)

ОТВЕТ : 
  1. Возражение про стоимость: - "Недешевое удовольствие." - "Как это может быть будет перерасчитываться, если я передумаю?" - "Я еще не готов потратить такие деньги на
 обучение." Предложения: Предложить клиенту рассмотреть более доступные опции обучения, например, пакет "база", который является более экономичным вариантом. Также можно
 предложить поговорить с куратором, чтобы он рассмотрел возможные варианты скидок или рассрочки платежа. 2. Возражение про время на обучение: - "Я очень занят и не
 уверен, что буду успевать." - "Я беспокоюсь о том, что обучение займет слишком много времени." Предложения: Предложить клиенту обсудить гибкий график обучения, который
 будет учитывать его занятость. Уточнить, насколько много времени клиент готов выделить на обучение, и предложить наиболее подходящие варианты курсов. 3. Возражение про
 негативные отзывы о компании и обучении: - "Я слышал некоторые негативные отзывы о ваших курсах." Предложения: Объяснить клиенту, что компания всегда

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, насколько проактивно менеджер действовал в диалоге',
    zoom_index
)

ОТВЕТ : 
  Отчет по проактивности менеджера в диалоге В данном диалоге можно отметить средний уровень проактивности со стороны менеджера. Проактивным моментом было предложение
 менеджером клиенту обучение на вакансии, где специалисты без опыта могут устроиться на работу. Также, менеджер предоставил клиенту возможность проходить обучение налогов
 быстрее, что так же можно отнести к проактивным действиям. Однако, в целом, менеджер в разговоре не использовал достаточно открытых вопросов, не проводил анализ
 интересов клиента, поэтому не всегда предлагал те решения, которые могли бы максимально соответствовать потребностям клиента. Таким образом, можно отметить, что менеджер
 мог быть более проактивен и активнее в предложении опций клиентам.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, в котором будут цитаты из текста, которые говорят о потребностях клиента',
    zoom_index
)

ОТВЕТ : 
  Отчет о потребностях клиента в курсах по AI разработке. Клиент заинтересован в форматах обучения: "Какие у нас есть? Формат обучения, да?" Клиент также хочет иметь
 возможность загружать фотографии и получать консультации: "не париться, не сидеть там в интернете... искусственный интеллект отслеживал их в поведении на дороге...
 Постере, а уже код мог предложить... и По поводу обучения могу разговаривать. Вдруг там что то будет вам не понятно или не ясно?" Клиент также хочет получить возможности
 по трудоустройству после обучения: "Там наши Чары полностью упаковывают ваш фильм. Кейс помогают сделать, так что как будто у вас опыта. Именно По годам больше, чем есть
 на самом деле... вы Из этих вакансий выбираете те, которые вам только интересны. Там два три. Делайте для них тестовые задания. Проходите, собеседование у них, и вам уже
 предлагают работу." Клиент также обеспокоен ценой курсов: "Ну, пока у меня нет такого. Как то? Недешевое удовольствие. Вот и тут надо взвешивать к

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Упоминал ли менеджер про возможность заморозки обучения?',
    zoom_index
)

ОТВЕТ : 
  Да, клиент спросил об условиях заморозки обучения, на что менеджер ответил: "Если не понравится, да и. Ну да. Или передумывает, По каким то причинам".


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Упоминал ли менеджер про возможность заморозки обучения? Если да, то приведи дословно цитату слов менеджера',
    zoom_index
)

ОТВЕТ : 
  Менеджер: Если не ошибаюсь, По законодательству такая опция есть у всех потребителей услуг. Если что, то вдруг. Мне нравится то вы можете прервать обучение.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Упоминал ли менеджер про возможность заморозки обучения? Не про полный отказ от обучения, а именно пр овременную заморозку. Если да, то приведи дословно цитату слов менеджера',
    zoom_index
)

ОТВЕТ : 
  Да, менеджер упоминал про возможность заморозки обучения. Дословная цитата менеджера: "Если не ошибаюсь, По законодательству такая опция есть у всех потребителей услуг.
 Если что, то вдруг. Мне нравится то вы можете прервать обучение."


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Упоминал ли менеджер сколько нужно времени на обучение? Если да, то приведи дословно цитату слов менеджера',
    zoom_index
)

ОТВЕТ : 
  Да, менеджер упоминал время на обучение. Дословная цитата: "На обучение я говорил минимум четыре часа. Желатину там десять часов в неделю."


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Упоминал ли менеджер зарплату начинающего специалиста после обучения? Если да, то приведи дословно цитату слов менеджера',
    zoom_index
)

ОТВЕТ : 
  Менеджер упоминал зарплату начинающего специалиста после обучения. Дословная цитата менеджера: "Зарплата вилка для начинающего специалиста от семьдесят до сто. Для медла
 уже от сто начинается."


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Какие примеры трудоустроенных выпускников приводил менеджер? Приведи дословно цитату слов менеджера',
    zoom_index
)

ОТВЕТ : 
  "Когда-то проекты на разработку. Поэтому наши студенты очень часто. Как объяснить? Вот дальнобойщики, которые часто там попадают. Конечную ситуацию на дорогах в аварии.
 Искусственный интеллект отслеживал их в поведении на дороге. Допустим, они отвлекают. То на телефон курили, застают и реагировал на это то есть предотвращение аварий. И
 вот наши студенты участвовали в таком проекте."


In [ ]:
zoom_promt_3 = '''Ты менеджер менеджер по продажам. Компания продает курсы по AI.
Тебе передали отчет по звонку с продажей клиенту, который совершал другой менеджер.
Тебе передают этого клиента на ведение и ты должен по отчету спланировать дальнейшую стратегию продажи этому клиенту.
'''

In [ ]:
gpt.get_chatgpt_ansver3(
    zoom_promt_3,
    '''Отчет:

Потребности клиента:
Отчет о потребностях клиента: Проанализировав диалог между менеджером и клиентом, можно выделить следующие потребности клиента: 1. Создание персонального шедевра через
 использование некоторых навыков, таких как вязание крючком. Клиент хочет определить свой стиль и подобрать уникальное изделие, которое будет модным. Также клиент
 интересовался возможностью оптимизации процесса создания персонального шедевра. 2. Понимание формата обучения. Клиент хотел привлекательный, интересный и необычный
 формат обучения, отличающийся от обычных методов. 3. Трудоустройство. Клиент интересовался процессом трудоустройства после обучения и возможностью увеличить свой опыт до
 необходимого для компаний уровня. Клиенту было предложено двадцать тридцать вакансий на выбор, с возможностью сделать для них тестовые задания и проходить собеседование.
 4. Использование искусственного интеллекта. В диалоге затрагивалась тема использования искусственного интеллекта в различных проектах, таких как отслеживание поведения
 дальнобойщиков на дороге для предотвращения аварий. 5. Различные проекты и задачи. Клиент был заинтересован в различных проектах, таких как автоматизированная
 расстановка сочельников на Посаде, разработка компьютерной программы для готовки документов, проектирование релиза в алгоритмах управления автомобилем и других. 6.
 Помощь в трудоустройстве. Компания предлагает помощь в трудоустройстве с помощью создания резюме и кейсов, что позволяет упаковать опыт клиента и увеличить его по годам.
 Таким образом, можно сделать вывод, что клиент заинтересован в уникальных и интересных форматах обучения, различных проектах и возможности трудоустройства после
 обучения. Также клиент интересуется использованием искусственного интеллекта и оптимизации процессов.

Возражения клиента:
Клиент не высказывал конкретных возражений, но задавал вопросы о цене и тарифах, а также интересовался, какие программы обучения предлагаются и возможности
 трудоустройства после курсов. Также клиент выразил свои сомнения относительно своей способности обучаться технологиям и проектам в сфере AI разработки, поскольку ранее
 не имел опыта работы в этой области. Тем не менее, клиент выразил интерес и готовность начать обучение с базовых программ и определенных тарифов обучения.
    

    
'''
)

ОТВЕТ : 
  Исходя из отчета, я рекомендую следующую стратегию продажи данному клиенту: 1. Демонстрация уникальных и интересных форматов обучения, которые предлагает компания,
 включая оригинальные проекты и различные возможности для обучения и развития. Это поможет клиенту понять, что компания предлагает не просто обычные курсы, а полноценный
 и уникальный опыт. 2. Объяснение тарифов и цен на курсы, а также предоставление конкретной информации о составе программ обучения, чтобы клиент мог лучше понимать то,
 что ему будет учиться и на что он тратит свои деньги. 3. Просмотр возможностей трудоустройства, которые предоставляет компания, в том числе предоставление конкретных
 вакансий в различных сферах, где требуются навыки в области AI. Это поможет клиенту понять, какую пользу ему может принести обучение в компании и как он может применить
 полученные знания на практике. 4. Предоставление конкретной информации о тестированиях и оказываемой поддержке после окончания обучения, чтобы клиент

<h3><b>Вычистка текста</b></h3>

In [ ]:
#Методичка
correct_index = gpt.load_prompt('https://docs.google.com/document/d/1h0Ujx_K0iXssQyykN2DIqypGsOtTj_Vcpe4NCJnk8NA/edit?usp=sharing')

In [ ]:
correct_promt = '''Ты копирайтер, который идеально умеет корректировать текст.
Тебе дают текст, который был распознан из аудио сервисом распознавания речи.
Сервис работает не идеально и допускает много ошибок.
Твоя задача восстановить текст максимально, насколько это возможно.
Тема текста - продажа клиенту обучения по AI.
Сохрани структуру с отметками, где говорит клиент, а где говорит менеджер.
Сохрани суть диалога для дальнейшей работы chatGPT с этим текстом.
Сохрани суть каждого сообщения, удаляй только те сообщения, в которых нет донесения смысла презентации.
Сделай вывод каждого сообщения клиента и менеджера с новой строчки.
''' 

In [ ]:
gpt.get_chatgpt_ansver3(
    correct_promt,
    'Скорректируй текст \n'+ correct_index
)

ОТВЕТ : 
  Диалог: Менеджер: Алло? Меня слышно? Менеджер: Алло, Ольга, слышно меня? Клиент: Да, да, слышу. Менеджер: Здравствуйте, Ольга. Меня зовут Лиана, очень приятно. Менеджер:
 Скажите, вы видите презентацию? Клиент: Да, я ее вижу. Менеджер: Хорошо, тогда скажу вам о наших целях. Менеджер: Про ваши цели говорим. Алло! Вы слышите меня? Клиент:
 Да, да. Менеджер: Хорошо, тогда продолжу. У нас есть короткая презентация по искусственному интеллекту с демонстрацией на выбор. Затем мы можем обсудить тарифы. Что
 наиболее интересует вас? Клиент: Я интересуюсь искусственным интеллектом и возможностями его использования. Менеджер: Какие именно возможности вас интересуют? Клиент: Я
 хотела бы узнать, как можно использовать искусственный интеллект в нашей компании. Менеджер: Понятно. Скажите, сколько времени свободного у вас есть в неделю для
 обучения? Клиент: У меня достаточно свободного времени. Менеджер: Хорошо, тогда начнем с нашей презентации. Мы можем заниматься проектами вместе и рас

In [ ]:
gpt.get_chatgpt_ansver3(
    correct_promt,
    '''Скорректируй текст 
    
Менеджер: Алло? Меня слышно?
Менеджер: Алло, слышно меня? Алло!
Менеджер: Алло, Ольга, меня слышно?
"Клиент: Да, да, слышно.
 Менеджер: Да. Здравствуйте, Ольга. Меня зовут лиана. Очень приятно с вами По."
"Менеджер: Скажите, видно вам презентацию?'''
)

ОТВЕТ : 
  Менеджер: Алло? Меня слышно? Менеджер: Алло, слышно меня? Алло! Менеджер: Алло, Ольга, меня слышно? Клиент: Да, да, я вас слышу. Менеджер: Здравствуйте, Ольга. Меня
 зовут Лиана. Очень приятно с вами поговорить. Менеджер: Скажите, вы видите на экране нашу презентацию?


In [ ]:
gpt.get_chatgpt_ansver3(
    correct_promt,
    '''Скорректируй текст 
    
Менеджер: Г. Хорошо. Тогда. Смотрите! Бак, получается, трудоустройство не интересует этот Филь. Мы просто хотели бы заниматься проектами. Может быть, там плановать, как то и где то на аутсорсе правила.
 Клиент: Да. Да. Расскажите, пожалуйста, Ольга, вообще, с чем у вас связан интерес к искусственному интеллекту?
 Клиент: На данный момент вот то, что я озвучила. Ещё придет. Чуть чуть потише. Ну как как что то современное, мне интересно.
 Менеджер: У.
 Клиент: Бы рассмотреть поближе.
 Менеджер: Что то современное? А подскажите вообще знакомых сойти там, может быть, занимались, чем то программировали языки знаете, какие то.
 Клиент: Да. Ну как бы мне сейчас сорок пять, когда я училась в институте.
 Менеджер: Угу. Г.'''
)

ОТВЕТ : 
  Менеджер: Хорошо. Тогда... Смотрите, получается, трудоустройство не интересует этого Фила. Мы просто хотели бы заниматься проектами. Может быть, там планировать, как-то
 и где-то на аутсорсе правила. Клиент: Да. Расскажите, пожалуйста, Ольга, вообще, с чем у вас связан интерес к искусственному интеллекту? Клиент: На данный момент - это
 то, что я озвучила. Ещё придет. Чуть-чуть потише... Мне интересно то, что является современным. Менеджер: Понятно. Клиент: Я бы рассмотрела поближе. Менеджер: Что-то
 современное? А подскажите, знакомы ли вам кто-то, кто занимался программированием, может быть, знаете какие-то языки программирования? Клиент: Да... Ну, как бы, мне
 сейчас 45 лет, когда я училась в институте... Менеджер: Понятно.


In [ ]:
gpt.get_chatgpt_ansver3(
    correct_promt,
    '''Скорректируй текст 
    
Менеджер: А. Вообще про интерес к искусственному интеллекту. Дальше будет короткое политанского университета. Демонстрацию на выбор. Вожди Того, что вас интересует больше всего и тарифы стоим, что обсудим хорошо."
"Клиент: Очень громко.
 Менеджер: А так?
 Клиент: У нас в процессе.
 Менеджер: Она в процессе. То есть вы уже открываете, хотели бы внедрить. Там искусственный интеллект заимст Того, как можно будет сделать.
 Клиент: Да.
 Менеджер: Угу. Всё получилось.'''
)

ОТВЕТ : 
  Менеджер: Здравствуйте! Я рад приветствовать вас! Вы выразили интерес к искусственному интеллекту. Я поделюсь информацией о нашем университете. Мы предлагаем краткую
 демонстрацию на выбор, чтобы вы могли определиться с тем, что вас интересует больше всего. Мы также обсудим тарифы и стоимость. "Клиент: Мне кажется, вы сейчас говорите
 очень громко. Менеджер: Извините, я поправлю громкость. Так лучше? Клиент: Да, спасибо. Менеджер: Я понимаю, что ваш проект находится в процессе. Вы уже открываете
 бизнес и хотите внедрить искусственный интеллект. Мы можем предложить вам несколько вариантов реализации. Хотели бы вы обсудить наши предложения? Клиент: Да, это было бы
 замечательно. Менеджер: Прекрасно, мы готовы ответить на все ваши вопросы и помочь вам в реализации проекта. Всё получилось?
